In [1]:
# for article content scripting with BeautifulSoup
import requests
from bs4 import BeautifulSoup
import re

# for rendering html template with Jinja
from jinja2 import Environment, FileSystemLoader
import os

# for building and sending emails with MailChimp
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError
import json

# for viewing the email draft in webpage
import webbrowser
from datetime import date

import config as cf
import importlib
cf = importlib.reload(cf)

# Define the Article Object with BeautifulSoup

In [19]:
class Article():
    
    def __init__(self, url, title=None, img_url=None, preview_text=None, tag=None, label=None, class_name='article', id_name='main', video_page_url=None):
        
        self._url = url.replace('/gb/', '/b5/')
        self._title = title
        self._img_url = img_url
        self._preview_text = preview_text
        self._tag = tag
        self._label = label
        self._video_page_url = video_page_url.replace('/gb/', '/b5/')
        
        page = requests.get(self._url)
        self._soup = BeautifulSoup(page.text, 'html.parser')
        
        self._article = self._soup.find(class_=class_name)
        
        if not self._article:
            self._article = self._soup.find(id=id_name)
            
        if not self._article:
            self._article = self._soup.find('article')
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        self._url = value
        
    @property
    def title(self, class_name='title'):
        
        if not self._title:
            article_title = self._article.find(class_=class_name)
            self._title = article_title.text
            
        return self._title
    
    @title.setter
    def title(self, value):
        self._title = value
    
    @property
    def img_url(self, class_name='featured_image'):
        
        if not self._img_url:
            try:
                self._img_url = self._article.find(class_=class_name).find('a').get('href')
            except:
                try:
                    self._img_url = self._article.find(class_=class_name).find('figure').find('img').get('src')
                except:
                    try:
                        self._img_url = self._article.find(class_='wp-post-image').get('src')
                    except:
                        page = requests.get(self._video_page_url)
                        video_page_soup = BeautifulSoup(page.text, 'html.parser')
                        self._img_url = video_page_soup.select_one(f"a[href='{self._url}']").find('img').get('src')
        
        return self._img_url
    
    @img_url.setter
    def img_url(self, value):
        self._img_url = value
    
    @property
    def preview_text(self, id_name='artbody'):
        
        paragraph_index = 0
        
        while not self._preview_text:
            article_first_paragraph = self._article.find(id=id_name).find_all('p')[paragraph_index].text
            article_preview_text = re.sub(r'\【[^)]*\】', '', article_first_paragraph)
            article_preview_text = re.sub(r'\n', '', article_preview_text) 
            article_preview_text = re.sub(r'\（英文大纪元[^)]*\编译）', '', article_preview_text)
            article_preview_text = re.sub(r'\（英文大紀元[^)]*\編譯）', '', article_preview_text)
            article_preview_text = re.sub(r'\（英國[^)]*\報導）', '', article_preview_text)
            article_preview_text = re.sub(r'\（大紀元[^)]*\報導）', '', article_preview_text)
            article_preview_text = re.sub(r'\（大紀元[^)]*\報導）', '', article_preview_text)
            self._preview_text = re.sub(r'\（大纪元[^)]*\报导）', '', article_preview_text)
            paragraph_index += 1
        
        return self._preview_text
    
    @preview_text.setter
    def preview_text(self, value):
        self._preview_text = value
        
    @property
    def tag(self, id_name='breadcrumb'):
        
        if not self._tag:
            breadcrumb = self._soup.find(id=id_name)
            raw_tag = breadcrumb.findAll('a')[-1].text
            raw_tag = re.sub(r'\([^)]*\)', '', raw_tag)
            self._tag = re.sub(r'\（[^)]*\）', '', raw_tag)
        
        return self._tag
    
    @tag.setter
    def tag(self, value):
        self._tag = value
        
    @property
    def label(self):
        return self._label
    
    @label.setter
    def label(self, value):
        self._label = value

In [20]:
class Content(Article):
    
    def __init__(self, article_url_list, video_url_list, video_page_url=None):
        
        # parse data
        self.article_list = [Article(article_url, video_page_url=video_page_url) for article_url in article_url_list]
        
        self.video_list = [Article(video_url, video_page_url=video_page_url) for video_url in video_url_list]
        self.video_pair_list = [[value, self.video_list[counter+1]] for counter, value in enumerate(self.video_list) if counter%2 == 0]      

In [21]:
class Jinja_Template(Content):
    
    def __init__(self, 
                 template_folder_name = 'Jinja Templates', 
                 template_file_name = 'newsletter_as_jinja_template_base.html'):

        file_loader = FileSystemLoader('../'+template_folder_name+'/')
        self.env = Environment(loader=file_loader, extensions=['jinja2.ext.do'])

        # render the base template
        self.template = self.env.get_template(template_file_name)
        
    def render(self, source_contect, 
               folder_name='Exported HTML', 
               rendered_newsletter_filename = 'rendered_newsletter.html'):
        
        # description: subject_line and preview_text
        self.subject_line = source_contect.article_list[0].title
        article_title_list = [article.title for article in source_contect.article_list]
        self.preview_text = ' · '.join(article_title_list[1:])
        
        self.rendered_content = self.template.render(env=self.env, 
                                                   header=source_contect.article_list[0], 
                                                   article_list=source_contect.article_list, 
                                                   video_pair_list=source_contect.video_pair_list)
        
        self.folder_path = '../' + folder_name + '/'
        
        if not os.path.exists(self.folder_path):
            os.makedirs(self.folder_path)
        
        self.newsletter_path = self.folder_path + rendered_newsletter_filename
        
        with open(self.newsletter_path, "w") as fh:
            fh.write(self.rendered_content)
            
        print('newsletter rendered at: '+ self.newsletter_path)
        
    def preview(self):

        webbrowser.open('file://' + os.path.realpath(self.newsletter_path))

In [22]:
class MailChimp_Campaign(Jinja_Template):

    def __init__(self, newsletter, mailchimp_config,
                 # sender information
                 from_name='Anonymous Sender',
                 reply_to='no_reply@email.com'):

        self.subject_line = newsletter.subject_line
        self.preview_text = newsletter.preview_text
        self.rendered_content = newsletter.rendered_content
        self.mailchimp_config = mailchimp_config

        # sender information
        self.from_name = from_name
        self.reply_to = reply_to
        
        # campaign name
        today = date.today().strftime("%d %B %Y")
        self.campaign_title = 'Campaign_Created_through_API' + today

    def connect_to_MailChimp(self):

        self.client = MailchimpMarketing.Client()
        self.client.set_config({
            "api_key": self.mailchimp_config["API_KEY"],
            "server": self.mailchimp_config["API_KEY"].split('-')[-1]
        })

        response_ping = self.client.ping.get()

        if response_ping['health_status'] == "Everything's Chimpy!":
            print("MailChimp account connected!")

    def update_MailChimp_template(self):

        response = self.client.templates.update_template(
            self.mailchimp_config['template_id'], {"name": "Template_Created_through_API", "html": self.rendered_content})
        print("Template updated at:", response['date_edited'])

    def specify_recipients(self):

        if self.mailchimp_config['segment_id']:
            self.recipients = {'segment_opts':
                               {'saved_segment_id':
                                   self.mailchimp_config['segment_id']},
                               'list_id': self.mailchimp_config['list_id']}
        else:
            self.recipients = {'list_id': self.mailchimp_config['list_id']}

        print('recipients specified.')

    def create(self):

        self.connect_to_MailChimp()
        self.update_MailChimp_template()
        self.specify_recipients()

        self.created_campaign = self.client.campaigns.create({"type": "regular",
                                                              'recipients': self.recipients,
                                                              "settings":
                                                              {"title": self.campaign_title,
                                                               "subject_line": self.subject_line,
                                                               "preview_text": self.preview_text,
                                                               "from_name": self.from_name,
                                                               "reply_to": self.reply_to,
                                                               "template_id": self.mailchimp_config['template_id'],
                                                               "auto_footer": False}})
        print('Campaign created successfully!')

    def send(self):

        campaign_id = self.created_campaign['id']
        response_of_campaigns_send = self.client.campaigns.send(campaign_id)

        return response_of_campaigns_send

# Processing Part

In [23]:
article_url_list = ['https://www.epochtimes.com/b5/21/5/4/n12923742.htm',
                    'https://www.epochtimes.com/b5/21/5/1/n12917316.htm',
                    'https://www.epochtimes.com/b5/21/5/5/n12927188.htm',
                    'https://www.epochtimes.com/b5/21/5/7/n12931073.htm']

video_url_list = ['https://www.epochtimes.com/gb/21/5/8/n12933638.htm',
                  'https://www.epochtimes.com/gb/21/5/7/n12932139.htm',
                  'https://www.epochtimes.com/gb/21/5/5/n12926621.htm',
                  'https://www.epochtimes.com/gb/21/5/8/n12933014.htm']

video_page_url = 'https://www.epochtimes.com/gb/ncid1247351.htm'

In [24]:
source_content = Content(article_url_list, video_url_list, video_page_url=video_page_url)

In [25]:
for article in source_content.article_list:
    print(article.title)
    print(article.tag)
    print(article.label)
    print(article.preview_text)
    print()

組圖：G7外長會議 英日外長舉行雙邊會談
國際要聞
None
七國集團（G7）外長會議目前於英國倫敦舉行，會期從5月3日至5月5日共三天。日本外務大臣茂木敏充於3日與英國外交大臣多米尼克・拉布（Dominic Raab）會晤，兩國舉行了雙邊會談。

兩位臺灣藝術家獲選，進駐倫敦戴芬娜基金會展開國際交流
教育新聞
None
中華民國（台灣）文化部駐英國代表處文化組與英國知名藝術機構戴芬娜基金會（Delfina Foundation）在今年合作公開遴選臺灣藝術家駐村的計劃，該會總監亞倫・西薩（Aaron Cezar）表示，由英國策展人、藝評家等專業人士組成的評選委員會對於臺灣藝術家的實力感到驚艷，決定增額邀請共兩位藝術家吳權倫及致穎，分別於夏季及秋季到英國，展開為期各3個月之駐村，期待協助臺灣新銳創作者與國際創作者交流，促成英國與歐陸策展人、藝評家，認識兩位藝術家的獨特創意及潛力。

首相府「裝修門」 監督部門正式調查
政治新聞
None
近日，有關英國首相約翰遜裝修首相府的資金來源問題，成為全國人的焦點。上週三（4月28日）監督部門「選舉委員會」（The Electoral Commission）宣佈正式對此展開調查。

組圖：歐聯盃半決賽次回合 曼聯總分8:5晉級
國際足球
None
5月6日，意大利羅馬，2020/21賽季歐聯盃半決賽次回合，英超曼徹斯特聯隊（簡稱「曼聯」）與意大利羅馬隊的比賽在奧林匹克球場（Stadio Olimpico）舉行，曼聯隊以2:3敗給羅馬隊。曼聯隊總分8:5勝羅馬隊，獲得晉級決賽的資格。



In [110]:
# source_content.article_list[0].title = '疫情导致去年英国经济缩水创300年记录'

In [26]:
for article in source_content.video_list:
    try:
        print(article.title)
        print(article.tag)
        print(article.preview_text)
        print(article.img_url)
        print()
    except:
        pass

【新聞看點】習加緊造神 高官知中共內情急退黨
新聞看點
大家好，歡迎關注新聞看點，我是李沐陽。今天是美東時間5月8日，星期六；亞洲時間是5月9日，星期日。
https://i.epochtimes.com/assets/uploads/2021/05/id12933656-0509-DJY-600x400.jpg

專訪李南央：我的兩本書《母親》和《繼母》（3）
熱點互動
特別節目，我是主持人方菲。本期節目我們再度請來李南央女士，來談一談她寫過的兩本很有影響的書。
https://i.epochtimes.com/assets/uploads/2021/05/id12935018-ttl7daypnY_ccdcfedf8de2a199.jpg

【探索時分】不要和美國開戰的五個理由（下）
軍事天地
大家好，我是子定，世界上總有些人想和美國開戰，我們上集介紹了不要和美國開戰的3個理由，不要和美國開戰的五個理由（上）。
https://i.epochtimes.com/assets/uploads/2021/05/id12926662-a8946dc1771cd08018c0b5f97f98033b-320x200.jpg

【唐浩視界】透視五大內幕 G7歐盟熱挺台灣
世界的十字路口
大家好，我是唐浩，今天都好嗎？
https://i.epochtimes.com/assets/uploads/2021/05/id12933041-959fe3a21766b8dfd6553b05448c5d3a-320x200.jpg



In [27]:
source_content.video_list[1].img_url

'https://i.epochtimes.com/assets/uploads/2021/05/id12935018-ttl7daypnY_ccdcfedf8de2a199.jpg'

In [28]:
for video in source_content.video_list:
    print(video.img_url)

https://i.epochtimes.com/assets/uploads/2021/05/id12933656-0509-DJY-600x400.jpg
https://i.epochtimes.com/assets/uploads/2021/05/id12935018-ttl7daypnY_ccdcfedf8de2a199.jpg
https://i.epochtimes.com/assets/uploads/2021/05/id12926662-a8946dc1771cd08018c0b5f97f98033b-320x200.jpg
https://i.epochtimes.com/assets/uploads/2021/05/id12933041-959fe3a21766b8dfd6553b05448c5d3a-320x200.jpg


In [29]:
source_content.video_list[0].img_url = 'https://i.ntdtv.com/assets/uploads/2021/04/maxresdefault-2-16-800x450.jpg'

In [36]:
source_content.video_list[1].img_url = 'https://i.epochtimes.com/assets/uploads/2021/04/id12871696-731c1292faf8ecb1db04b660aa2871c3-600x400.jpg'

In [37]:
source_content.video_list[2].img_url = 'https://i.epochtimes.com/assets/uploads/2021/04/id12871426-a948bfecbdb0af2e8292697829ecb2a5-320x200.jpg'

In [38]:
source_content.video_list[3].img_url = 'https://i.epochtimes.com/assets/uploads/2021/04/id12871886-51960545c5e2f1bfd1522d9ccab9f3d5-320x200.jpg'

In [29]:
newsletter = Jinja_Template()

In [30]:
newsletter.render(source_content)

newsletter rendered at: ../Exported HTML/rendered_newsletter.html


In [31]:
newsletter.preview()

# Connect to MailChimp

## Test Campaign

In [32]:
mailchimp_config_test = {
    "API_KEY": cf.mailchimp_config_test.API_KEY,
    "template_id": cf.mailchimp_config_test.template_id,
    "segment_id": cf.mailchimp_config_test.segment_id, # recipients
    "list_id": cf.mailchimp_config_test.list_id # members list
}

### Create MailChimp campaign

In [33]:
test_campaign = MailChimp_Campaign(newsletter, mailchimp_config_test, 
                                   from_name=cf.mailchimp_compaign.from_name, 
                                   reply_to=cf.mailchimp_compaign.reply_to)

In [34]:
test_campaign.create()

MailChimp account connected!
Template updated at: 2021-05-09T18:59:13+00:00
recipients specified.
Campaign created successfully!


In [35]:
test_campaign.send()

<Response [204]>

## Real Campaign

In [36]:
mailchimp_config_real = {
    "API_KEY": cf.mailchimp_config_real.API_KEY,
    "template_id": cf.mailchimp_config_real.template_id,
    "segment_id": cf.mailchimp_config_real.segment_id, # recipients
    "list_id": cf.mailchimp_config_real.list_id # members list
}

In [37]:
real_campaign = MailChimp_Campaign(newsletter, mailchimp_config_real, 
                                   from_name=cf.mailchimp_compaign.from_name, 
                                   reply_to=cf.mailchimp_compaign.reply_to)

In [38]:
real_campaign.create()

MailChimp account connected!
Template updated at: 2021-05-09T18:59:53+00:00
recipients specified.
Campaign created successfully!


In [39]:
real_campaign.send()

<Response [204]>

## Drafts

In [200]:
# define article tag
tag_list = ['英国新闻', '英国新闻', '英国新闻', '英国新闻']

for index in range(len(tag_list)):
    article_list[index].tag = tag_list[index]
    
# article_list[0].img_url = 'https://i.epochtimes.com/assets/uploads/2020/12/2012112056442124.jpg'